In [2]:
#lista de parametros para API

import requests
import pandas as pd
import numpy as np

response = requests.get('https://servicodados.ibge.gov.br/api/v3/agregados')

if response.status_code == 200:
    data = response.json()

    resultado = pd.DataFrame({'categ_id':[], 'categ_nome':[], 'categ_unidade':[], 'categ_nivel':[], 'tabela_id':[],
       'tabela_classificacoes.id':[], 'tabela_classificacoes.nome':[],                      
       'var_id':[], 'var_nome':[], 'var_unidade':[], 'var_sumarizacao':[], 'tabela_id':[],
       'Tabela_nome':[], 'Tabela_URL':[], 'Tabela_pesquisa':[], 'Tabela_assunto':[],
       'Tabela_periodicidade.frequencia':[], 'Tabela_periodicidade.inicio':[],
       'Tabela_periodicidade.fim':[], 'Tabela_nivelTerritorial.Administrativo':[]})
    
  
    lista = pd.DataFrame({'tabela':['995']})
    print(lista['tabela'])
    for item in lista['tabela']:
        tab = item
        #print(IdTabela)
        enderecoT = 'https://servicodados.ibge.gov.br/api/v3/agregados/'+tab+'/metadados'
        print(enderecoT)
        responseT = requests.get(enderecoT)
        dataT = responseT.json()
        #print(dataT)
        df_categ = pd.json_normalize(dataT,
            record_path=['classificacoes','categorias'],
            meta=["id",["classificacoes","id"],["classificacoes","nome"]],
            meta_prefix='tabela_',
            record_prefix='categ_'
            )
       # print(df_categ)
                #Variaveis x classificacao

        df_var = pd.json_normalize(dataT,
            record_path=['variaveis'],
            meta=["id","nome","URL","pesquisa","assunto",["periodicidade","frequencia"],["periodicidade","inicio"],["periodicidade","fim"],["nivelTerritorial","Administrativo"]],
            meta_prefix='tabela_',
            record_prefix='var_'
                    )
        #nivel territorio x tabela

        df_nivel = pd.json_normalize(dataT,
            record_path=['nivelTerritorial','Administrativo'],
            meta=["id"],
            meta_prefix='tabela_',
            record_prefix='nivel_'
            )

        if df_categ.empty:
            dfm = df_var.merge(df_nivel,left_on='tabela_id',right_on='tabela_id')
        else:
            df1 = df_categ.merge(df_var, left_on='tabela_id',right_on='tabela_id')
            dfm = df1.merge(df_nivel, left_on='tabela_id',right_on='tabela_id')
        
        options = ['N1','N3','N6'] 
        df = dfm.loc[dfm['nivel_0'].isin(options)] 
        #df = df.loc[:,'tabela_id']#,'var_id','tabela_classificacoes.id','categ_id','nivel_0']]
        #print(dfm)
        #print("muda/muda/muda")
        resultado = pd.concat([resultado,df])
        resultado = resultado.loc[:,['tabela_id','var_id','tabela_classificacoes.id','categ_id','nivel_0']]
        resultado[['tabela_id','var_id','tabela_classificacoes.id','categ_id']] = resultado[['tabela_id','var_id','tabela_classificacoes.id','categ_id']].fillna(-1)
        resultado[['tabela_id','var_id','tabela_classificacoes.id','categ_id']] = resultado[['tabela_id','var_id','tabela_classificacoes.id','categ_id']].astype(int)
        resultado[['tabela_id','var_id','tabela_classificacoes.id','categ_id']] = resultado[['tabela_id','var_id','tabela_classificacoes.id','categ_id']].astype(str)
        resultado[['tabela_id','var_id','tabela_classificacoes.id','categ_id']] = resultado[['tabela_id','var_id','tabela_classificacoes.id','categ_id']].replace('-1', np.nan)
        resultado[['tabela_id','var_id','tabela_classificacoes.id','categ_id']] = resultado[['tabela_id','var_id','tabela_classificacoes.id','categ_id']].fillna('').astype(str)

    print("RESULTADO/RESULTADO/RESULTADO")    
    print(resultado.dtypes)
    print(resultado)
    print(f"Request failed with status code {response.status_code}")

0    995
Name: tabela, dtype: object
https://servicodados.ibge.gov.br/api/v3/agregados/995/metadados
RESULTADO/RESULTADO/RESULTADO
tabela_id                   object
var_id                      object
tabela_classificacoes.id    object
categ_id                    object
nivel_0                     object
dtype: object
     tabela_id var_id tabela_classificacoes.id categ_id nivel_0
0          995    630                    12762   116830      N1
1          995    707                    12762   116830      N1
2          995   1637                    12762   116830      N1
3          995   1636                    12762   116830      N1
4          995   1639                    12762   116830      N1
...        ...    ...                      ...      ...     ...
2401       995    707                    12388   107695      N1
2402       995   1637                    12388   107695      N1
2403       995   1636                    12388   107695      N1
2404       995   1639                   

C:\Users\Alexandre Rebello\AppData\Local\Temp\ipykernel_32332\1823616759.py:67: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  resultado[['tabela_id','var_id','tabela_classificacoes.id','categ_id']] = resultado[['tabela_id','var_id','tabela_classificacoes.id','categ_id']].fillna(-1)
